In [ ]:
from os import path

import astropy.coordinates as coord
from astropy.table import Table
import astropy.units as u
from astropy.io import ascii
from astropy.io import fits
from astropy.wcs import WCS
import reproject

import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline
import h5py
from scipy.spatial import cKDTree

from pyia import GaiaData
import gala.coordinates as gc
from dustmaps.sfd import SFDQuery
from astroquery.gaia import Gaia

from sfd import reddening

# Gaia.login(user='apricewh', password='7Gy2^otQNj6FKz6')

In [ ]:
lmc_c = coord.SkyCoord.from_name('LMC')
smc_c = coord.SkyCoord.from_name('SMC')

In [ ]:
q_fmt ='''SELECT gaia.source_id, gaia.ra, gaia.dec, gaia.parallax, gaia.pmra, gaia.pmdec, gaia.radial_velocity, 
gaia.phot_g_mean_mag, gaia.phot_bp_mean_mag, gaia.phot_rp_mean_mag, 
ps1.g_mean_psf_mag, ps1.r_mean_psf_mag, ps1.i_mean_psf_mag, ps1.z_mean_psf_mag, ps1.y_mean_psf_mag
FROM gaiadr2.gaia_source AS gaia
INNER JOIN gaiadr2.panstarrs1_best_neighbour AS xmatch
    ON gaia.source_id = xmatch.source_id
INNER JOIN gaiadr2.panstarrs1_original_valid AS ps1
    ON ps1.obj_id = xmatch.original_ext_source_id
WHERE gaia.parallax < 1 AND 
    (ps1.g_mean_psf_mag - ps1.i_mean_psf_mag) > -1 AND 
    (ps1.g_mean_psf_mag - ps1.i_mean_psf_mag) < 1 AND 
    CONTAINS(POINT('ICRS', gaia.ra, gaia.dec), 
             CIRCLE('ICRS', {0.ra.degree}, {0.dec.degree}, {1})) = 1 AND
    (pmra > -10) AND (pmra < 10) AND 
    (pmdec > -10) AND (pmdec < 10) 
'''

In [ ]:
c = coord.SkyCoord(ra=179, dec=-29, unit=u.deg)

In [ ]:
print(q_fmt.format(c, 3.))

In [ ]:
fn = '../data/lmc-LAII-field.fits'
if not path.exists(fn):
    # job = Gaia.launch_job_async(q)
    tbl = job.get_results()
    tbl.write(fn)
    
g = GaiaData(fn)

In [ ]:
c = g.get_skycoord(distance=False)
A_grizy = reddening(c, 'PS1', 'grizy')

In [ ]:
mag = dict()
for i, m in enumerate('grizy'):
    mag[m] = np.array(getattr(g, m + '_mean_psf_mag')) - A_grizy[:, i]

In [ ]:
gi_g = np.vstack((mag['g']-mag['i'], mag['g'])).T
rz_r = np.vstack((mag['r']-mag['z'], mag['r'])).T

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(10, 5), sharey=True)

axes[0].plot(gi_g[:, 0], gi_g[:, 1],
             marker='.', linestyle='none', alpha=0.8)
axes[0].set_xlabel('')
axes[0].set_ylabel('')
axes[0].set_xlim(-1, 1)
axes[0].set_ylim(21, 12)

gi_nodes = np.array([[-0.6, 16],
                     [-0.4, 18.7],
                     [0, 20],
                     [0, 21],
                     [-0.5, 20],
                     [-0.7, 19],
                     [-0.75, 17.],
                     [-0.8, 16]])
gi_path = mpl.patches.Path(gi_nodes)
axes[0].plot(gi_nodes[:,0], gi_nodes[:,1])

axes[1].plot(rz_r[:, 0], rz_r[:, 1],
             marker='.', linestyle='none', alpha=0.8)
axes[1].set_xlabel('')
axes[1].set_ylabel('')
axes[1].set_xlim(-1, 1)
axes[1].set_ylim(21, 12)

X = np.vstack((mag['g']-mag['i'], mag['g'])).T
gi_mask = gi_path.contains_points(X)

axes[0].set_xlabel(r'$g_{\rm PS1} - i_{\rm PS1}$')
axes[0].set_ylabel(r'$g_{\rm PS1}$')

axes[1].set_xlabel(r'$r_{\rm PS1} - z_{\rm PS1}$')
axes[1].set_ylabel(r'$r_{\rm PS1}$')

fig.tight_layout()
# fig.savefig('/Users/adrian/Downloads/cmd.png', dpi=250)

In [ ]:
# cmd_mask = ((mag['g']-mag['i']) < 0.) & ((mag['g']-mag['i']) > -0.5)
cmd_mask = gi_mask

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(12, 5.5))

axes[0].plot(g.ra, g.dec,
             marker='.', linestyle='none', 
             alpha=0.1, color='#aaaaaa')
axes[0].plot(g.ra[cmd_mask], g.dec[cmd_mask],
             marker='.', linestyle='none', alpha=1)
axes[0].set_xlabel('RA')
axes[0].set_ylabel('Dec')

axes[1].plot(g.pmra, g.pmdec,
             marker='.', linestyle='none', 
             alpha=0.4, color='#aaaaaa')
axes[1].plot(g.pmra[cmd_mask], g.pmdec[cmd_mask],
             marker='.', linestyle='none', alpha=1.)

axes[1].set_xlim(-5, 5)
axes[1].set_ylim(-5, 5)
axes[1].set_xlabel('pmra')
axes[1].set_ylabel('pmdec')

fig.tight_layout()

In [ ]:
pm_mask = np.sqrt((g.pmra - -0.5*u.mas/u.yr)**2 + (g.pmdec - 0.4*u.mas/u.yr)**2) < 0.7*u.mas/u.yr

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(12, 5.5))

axes[0].plot(g.ra, g.dec,
             marker='.', linestyle='none', 
             alpha=0.1, color='#aaaaaa')
axes[0].plot(g.ra[cmd_mask & pm_mask], g.dec[cmd_mask & pm_mask],
             marker='.', linestyle='none', alpha=1)
axes[0].set_xlabel('RA')
axes[0].set_ylabel('Dec')

axes[1].plot(g.pmra, g.pmdec,
             marker='.', linestyle='none', 
             alpha=0.4, color='#aaaaaa')
axes[1].plot(g.pmra[cmd_mask & pm_mask], g.pmdec[cmd_mask & pm_mask],
             marker='.', linestyle='none', alpha=1.)

axes[1].set_xlim(-5, 5)
axes[1].set_ylim(-5, 5)
axes[1].set_xlabel('pmra')
axes[1].set_ylabel('pmdec')

fig.tight_layout()

In [ ]:
np.nanmean(g.parallax[cmd_mask & pm_mask])
plt.hist(g.parallax[cmd_mask & pm_mask], bins=np.linspace(-0.5, 0.5, 16));

Load LMC stream gas data from Nidever+2008

In [ ]:
hdu = fits.open('/Users/adrian/data/Misc/allms_coldens_log.fits.gz')[0]

In [ ]:
wcs = WCS(hdu.header)

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(8, 6), subplot_kw=dict(projection=wcs))
ax.imshow(hdu.data, origin='lower', )
fig.tight_layout()

In [ ]:
hdr = hdu.header.copy()
hdr['CRVAL1'] = 40
wcs2 = WCS(hdr)

In [ ]:
arr, _ = reproject.reproject_interp(hdu, wcs2, shape_out=(1601, 3000))

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(8, 6), subplot_kw=dict(projection=wcs2))

# ax = fig.add_subplot(gs[0, :], projection=wcs2)
ax.imshow(arr, origin='lower', cmap='Greys', vmin=15, vmax=25)
# ax.scatter(60, -20, transform=ax.get_transform('world'))

# ---
# ax = fig.add_subplot(gs[1, 0], projection=wcs2)

derp = c.transform_to(gc.MagellanicStream)
ax.plot(derp.L.degree[cmd_mask & pm_mask], derp.B.degree[cmd_mask & pm_mask],
        marker='.', linestyle='none', alpha=0.5,
        transform=ax.get_transform('world'))
# ax.set_xlabel('RA')
# ax.set_ylabel('Dec')

fig.tight_layout()

# fig.savefig('/Users/adrian/Downloads/LMC-coords.png', dpi=250)

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(10, 5), sharey=True)

axes[0].plot(gi_g[pm_mask, 0], gi_g[pm_mask, 1],
             marker='.', linestyle='none', alpha=0.8)
axes[0].set_xlabel('')
axes[0].set_ylabel('')
axes[0].set_xlim(-1, 0.75)
axes[0].set_ylim(21, 12)

axes[1].plot(rz_r[pm_mask, 0], rz_r[pm_mask, 1],
             marker='.', linestyle='none', alpha=0.8)
axes[1].set_xlabel('')
axes[1].set_ylabel('')
axes[1].set_xlim(-1, 0.5)
axes[1].set_ylim(21, 12)

axes[0].set_xlabel(r'$g_{\rm PS1} - i_{\rm PS1}$')
axes[0].set_ylabel(r'$g_{\rm PS1}$')

axes[1].set_xlabel(r'$r_{\rm PS1} - z_{\rm PS1}$')
axes[1].set_ylabel(r'$r_{\rm PS1}$')

fig.tight_layout()

In [ ]:
iso = Table.read('../data/MIST_iso_5afc2f0778826.iso.cmd', header_start=12, format='ascii')

In [ ]:
DM = coord.Distance(25*u.kpc).distmod.value

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(10, 5), sharey=True)

axes[0].plot(gi_g[pm_mask, 0], gi_g[pm_mask, 1],
             marker='.', linestyle='none', alpha=0.8)
axes[0].set_xlabel('')
axes[0].set_ylabel('')
axes[0].set_xlim(-1, 0.75)
axes[0].set_ylim(21, 12)

axes[1].plot(rz_r[pm_mask, 0], rz_r[pm_mask, 1],
             marker='.', linestyle='none', alpha=0.8)
axes[1].set_xlabel('')
axes[1].set_ylabel('')
axes[1].set_xlim(-1, 0.5)
axes[1].set_ylim(21, 12)

axes[0].set_xlabel(r'$g_{\rm PS1} - i_{\rm PS1}$')
axes[0].set_ylabel(r'$g_{\rm PS1}$')

axes[1].set_xlabel(r'$r_{\rm PS1} - z_{\rm PS1}$')
axes[1].set_ylabel(r'$r_{\rm PS1}$')

# --- isochrones ---
for age in np.unique(iso['isochrone_age_yr'])[:4]:
    age_mask = iso['isochrone_age_yr'] == age
    phase_mask = (iso['phase'] >= 0) & (iso['phase'] < 3)
    sub_iso = iso[age_mask & phase_mask]    
    
    axes[0].plot(sub_iso['PS_g']-sub_iso['PS_i'], 
                 sub_iso['PS_g'] + DM,
                 marker='', alpha=0.5, lw=2)
    
    axes[1].plot(sub_iso['PS_r']-sub_iso['PS_z'], 
                 sub_iso['PS_r'] + DM,
                 marker='', label=r'${:.0f}\,{{\rm Myr}}$'.format(age/1e6),
                 alpha=0.5, lw=2)
    
axes[1].legend(loc='upper left')

fig.tight_layout()

fig.savefig('../plots/isochrone.png', dpi=250)

In [ ]:
from spectral_cube import SpectralCube

In [ ]:
hdu = fits.open('/Users/adrian/Downloads/gass_289_32_1526493902.fits.gz')[0]

In [ ]:
hdr = hdu.header
del hdr['CUNIT3']

In [ ]:
WCS(hdr)

In [ ]:
hdu = fits.open('/Users/adrian/Downloads/gass_289_32_1526493902.fits.gz')[0]
del hdu.header['CUNIT3']
cube = SpectralCube.read(hdu)

In [ ]:
cube[100,:,:].quicklook()

In [ ]:
icrs = coord.SkyCoord(ra=g.ra[cmd_mask & pm_mask], 
                      dec=g.dec[cmd_mask & pm_mask])
gal = icrs.transform_to(coord.Galactic)

In [ ]:
sub_cube_slab = cube.spectral_slab(240*u.km/u.s, 260*u.km/u.s)
sub_cube = sub_cube_slab.max(axis=0)

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(6, 6), 
                       subplot_kw=dict(projection=sub_cube.wcs))

# Display the slice
im = ax.imshow(sub_cube.hdu.data, origin='lower', cmap='Greys_r', 
               vmin=-0.2, vmax=2)

ax.scatter(gal.l.degree, gal.b.degree, 
           transform=ax.get_transform('galactic'),
           marker='.')

# # Add axes labels
# ax.set_xlabel("LSR Velocity (m/s)", fontsize = 16)
# ax.set_ylabel("Galactic Longitude (degrees)", fontsize = 16)

fig.savefig('../plots/GASS.png', dpi=250)

In [ ]:
icrs = coord.SkyCoord(ra=g.ra[cmd_mask & pm_mask], 
                      dec=g.dec[cmd_mask & pm_mask],
                      distance=25*u.kpc,
                      pm_ra_cosdec=g.pmra[cmd_mask & pm_mask], 
                      pm_dec=g.pmdec[cmd_mask & pm_mask],
                      radial_velocity=250*u.km/u.s)

In [ ]:
galcen = icrs.transform_to(coord.Galactocentric)

In [ ]:
import gala.coordinates as gc
import gala.dynamics as gd
import gala.integrate as gi
import gala.potential as gp
from gala.units import galactic

In [ ]:
w0 = gd.PhaseSpacePosition(galcen.cartesian)
pot = gp.MilkyWayPotential()

In [ ]:
orbit = pot.integrate_orbit(w0, dt=-0.25, n_steps=250)
fig = orbit.cylindrical.plot(['rho', 'z'], color='k', alpha=0.1)
fig.axes[0].set_xlim(0, 30)
fig.axes[0].set_ylim(-15, 15)
fig.axes[0].axhline(0, zorder=-1, color='tab:blue')
print(orbit.t[-1])
fig.savefig('../plots/orbits.png', dpi=250)